In [46]:
import pandas as pd
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score

def convert_label(value_map):
    def mapper(label):
        if label in value_map:
            return value_map[label]
        else:
            return label
    return mapper


In [47]:
# Load the data into a pandas DataFrame
df = pd.read_excel('az10.xlsx')

In [49]:
data = df

In [50]:
data.head()


,Well ID,X,Y,DEPTH,CALI,DT,NPHI,PHIE,RHOB,SW,litho,VANH,VCLC,VDOL,VOL_QUARTZ,VOL_SHALE,Un
0,AZ-010,607255.5255,2054041.255,2555.1384,5.7268,61.7078,0.1010,0.1201,2.5714,0.3912,LIMESTONE,0.1401,0.6317,0.1080,0,0.0,-999.25
1,AZ-010,607255.5255,2054041.255,2555.2908,5.7468,65.2759,0.1378,0.1480,2.5083,0.2880,LIMESTONE,0.0288,0.5921,0.2312,0,0.0,-999.25
2,AZ-010,607255.5255,2054041.255,2555.4432,5.8124,68.7534,0.1657,0.1730,2.4661,0.2421,LIMESTONE,0.0346,0.5207,0.2717,0,0.0,-999.25
3,AZ-010,607255.5255,2054041.255,2555.5956,5.8405,70.1109,0.1981,0.1984,2.4371,0.2994,MIX,0.0034,0.3592,0.4391,0,0.0,-999.25
4,AZ-010,607255.5255,2054041.255,2555.7480,5.7661,73.7412,0.2272,0.2284,2.4057,0.3439,DOLOMITE,0.0001,0.2274,0.5441,0,0.0,-999.25


In [51]:
unique_values = data['litho'].unique()
unique_values

array(['LIMESTONE', 'MIX', 'DOLOMITE', 'ANHYDRIDE', 'SANDSTONE', 'SHALE'],
      dtype=object)

In [52]:

data["litho"] = data["litho"].apply(convert_label({'LIMESTONE': 0, 'MIX': 1,'SANDSTONE':2,'DOLOMITE':4, 'ANHYDRIDE':5, 'SHALE':6}))
data = data.drop('Well ID', axis=1)
data = data.drop('VOL_QUARTZ', axis=1)
data = data.drop('Un', axis=1)
data.head()


,X,Y,DEPTH,CALI,DT,NPHI,PHIE,RHOB,SW,litho,VANH,VCLC,VDOL,VOL_SHALE
0,607255.5255,2054041.255,2555.1384,5.7268,61.7078,0.1010,0.1201,2.5714,0.3912,0,0.1401,0.6317,0.1080,0.0
1,607255.5255,2054041.255,2555.2908,5.7468,65.2759,0.1378,0.1480,2.5083,0.2880,0,0.0288,0.5921,0.2312,0.0
2,607255.5255,2054041.255,2555.4432,5.8124,68.7534,0.1657,0.1730,2.4661,0.2421,0,0.0346,0.5207,0.2717,0.0
3,607255.5255,2054041.255,2555.5956,5.8405,70.1109,0.1981,0.1984,2.4371,0.2994,1,0.0034,0.3592,0.4391,0.0
4,607255.5255,2054041.255,2555.7480,5.7661,73.7412,0.2272,0.2284,2.4057,0.3439,4,0.0001,0.2274,0.5441,0.0


In [53]:
# Get the number of rows and columns
num_rows, num_cols = data.shape
print(f"The DataFrame has {num_rows} rows and {num_cols} columns.")

The DataFrame has 2302 rows and 14 columns.


In [96]:
# train = data[10000:10010]
train = data.sample(n=500, random_state=74)
test = data.drop(train.index)

# Assuming your DataFrame is named 'train' and the target column is named 'target'
X = train.drop('litho', axis=1).values
y = train['litho'].values.reshape(-1, 1)
y = y.ravel()

data.head()

,X,Y,DEPTH,CALI,DT,NPHI,PHIE,RHOB,SW,litho,VANH,VCLC,VDOL,VOL_SHALE
0,607255.5255,2054041.255,2555.1384,5.7268,61.7078,0.1010,0.1201,2.5714,0.3912,0,0.1401,0.6317,0.1080,0.0
1,607255.5255,2054041.255,2555.2908,5.7468,65.2759,0.1378,0.1480,2.5083,0.2880,0,0.0288,0.5921,0.2312,0.0
2,607255.5255,2054041.255,2555.4432,5.8124,68.7534,0.1657,0.1730,2.4661,0.2421,0,0.0346,0.5207,0.2717,0.0
3,607255.5255,2054041.255,2555.5956,5.8405,70.1109,0.1981,0.1984,2.4371,0.2994,1,0.0034,0.3592,0.4391,0.0
4,607255.5255,2054041.255,2555.7480,5.7661,73.7412,0.2272,0.2284,2.4057,0.3439,4,0.0001,0.2274,0.5441,0.0


In [97]:
mlp = MLPClassifier(hidden_layer_sizes=(2,), random_state=42)

mlp.fit(X, y)

from sklearn.metrics import accuracy_score

X_test = test.iloc[:, :-1].values
Y_test = test.iloc[:, -1].values.reshape(-1,1)

# Convert the target variable to a binary variable
Y_test = (Y_test > 0.5).astype(int)  # Assuming the target variable is a continuous variable

y_pred = mlp.predict(X_test)


accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9927857935627081


# find Best accuracy

In [98]:
import itertools

num_layers = 3
max_neurons = 30

layer_combinations = []
# for r in range(1, num_layers + 1):
#     layer_combinations += list(itertools.product(range(1, max_neurons + 1), repeat=r))


In [99]:
best_accuracy = 0
best_arrangement = None

for arrangement in layer_combinations:
    mlp = MLPClassifier(hidden_layer_sizes=arrangement, random_state=42)

    mlp.fit(X, y)
    y_pred = mlp.predict(X_test)

    accuracy = accuracy_score(Y_test, y_pred)
    if accuracy > 0.80:
        print("Arrangement:", arrangement)
        print("Accuracy:", accuracy)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_arrangement = arrangement

print("Best Arrangement:", best_arrangement)
print("Best Accuracy:", best_accuracy)

Best Arrangement: None
Best Accuracy: 0
